# Google Play Store Reviews

## Import Libraries

In [1]:
import pandas as pd
import os

# Import Google Play Store Reviews Data
from google_play_scraper import Sort, reviews_all

## Data Collection

In [2]:
PlayStoreData = pd.read_csv("GamesData.csv")
NotGamesCat = 'Art & Design|Auto & Vehicles|Beauty|Books & Reference|Business|Comics|Communication|Dating|Education|Entertainment|Events|Finance|Food & Drink|Health & Fitness|House & Home|Libraries & Demo|Lifestyle|Maps & Navigation|Medical|Music|Music & Audio|News & Magazines|Parenting|Personalization|Photography|Productivity|Shopping|Social|Tools|Travel & Local|Video Players & Editors|Weather'
PlayStoreGames = PlayStoreData.sort_values(by='Rating Count', ascending=False)[~PlayStoreData['Category'].str.contains(NotGamesCat)]
print(f"Found {PlayStoreGames.index.size} Games in the Google Play Store")
PlayStoreGames

C:\Users\Austin\AppData\Local\Temp\ipykernel_12812\2760600622.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  PlayStoreGames = PlayStoreData.sort_values(by='Rating Count', ascending=False)[~PlayStoreData['Category'].str.contains(NotGamesCat)]


Found 350486 Games in the Google Play Store


,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
244319,Garena Free Fire - Rampage,com.dts.freefireth,Action,4.2,89177097.0,"500,000,000+",5.000000e+08,976536041,True,0.0,...,https://ff.garena.com,freefire@garena.com,"Dec 7, 2017","Jun 04, 2021",Mature 17+,https://ff.garena.com/others/policy/en/,False,True,True,2021-06-16 00:32:12
423997,Clash of Clans,com.supercell.clashofclans,Strategy,4.5,56025424.0,"500,000,000+",5.000000e+08,643789632,True,0.0,...,http://supercell.helpshift.com/a/clash-of-clans/,gp-info@supercell.com,"Sep 30, 2013","Jun 09, 2021",Everyone 10+,http://www.supercell.net/privacy-policy,False,True,True,2021-06-16 03:26:52
58082,PUBG MOBILE - Traverse,com.tencent.ig,Action,4.3,37479011.0,"500,000,000+",5.000000e+08,505818718,True,0.0,...,NaN,PUBGMOBILE_CS@tencentgames.com,"Mar 19, 2018","May 10, 2021",Teen,http://pubgmobile.proximabeta.com/privacy.html,True,True,True,2021-06-15 21:17:17
286345,Candy Crush Saga,com.king.candycrushsaga,Casual,4.6,31476637.0,"1,000,000,000+",1.000000e+09,1208422684,True,0.0,...,http://candycrushsaga.com/help/,candycrush.techhelp@king.com,"Nov 15, 2012","Jun 04, 2021",Everyone,https://king.com/privacyPolicy,True,True,False,2021-06-16 01:14:37
1433519,Clash Royale,com.supercell.clashroyale,Strategy,4.2,31018623.0,"100,000,000+",1.000000e+08,405849099,True,0.0,...,http://supercell.helpshift.com/a/clash-royale/,gp-info@supercell.com,"Mar 1, 2016","Jun 10, 2021",Everyone 10+,http://www.supercell.net/privacy-policy,False,True,True,2021-06-16 00:21:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2311821,DreamWorks Universe of Legends,com.firefly.dreamworks.dwuol,Role Playing,NaN,NaN,NaN,NaN,292415,False,0.0,...,http://www.fireflygames.com/,cs@fireflygames.com,NaN,"May 23, 2019",Everyone,http://www.fireflygames.com/privacy_policy.html,False,False,False,2021-06-16 12:58:20
2312319,Reptile match,com.ourexotics.reptilematch,Puzzle,NaN,NaN,10+,1.000000e+01,39,True,0.0,...,http://www.OurExotics.com,allreptiles200@gmail.com,NaN,"Jan 20, 2016",Everyone,http://www.ourexotics.com/privacy.aspx,False,False,False,2021-06-16 12:58:45
2312545,Critical Cover Hunter Ops:Free Shooting Games ...,com.tgames.offline.shooter.black.strike.ops.fp...,Role Playing,NaN,NaN,"1,000+",1.000000e+03,2275,True,0.0,...,NaN,taggames2020@gmail.com,NaN,"Mar 18, 2021",Teen,https://sites.google.com/view/topactiongamesta...,True,False,False,2021-06-16 12:58:57
2312553,Merlins Idle Apelsin,com.elitegamesltd.merlinsidle,Simulation,NaN,NaN,10+,1.000000e+01,44,True,0.0,...,NaN,efgames.net@gmail.com,NaN,"Nov 17, 2019",Everyone,http://theelitegames.net/privacy.html,False,False,False,2021-06-16 12:58:58


In [3]:
TopGames = PlayStoreGames[['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Released', 'Content Rating']][:20].copy()
TopGames = TopGames.sort_values(by='App Name')
TopGames['App Name'] = TopGames['App Name'].str.split(' - ').str[0]
TopGames['App Name'] = TopGames['App Name'].str.split(':').str[0]
TopGames.to_csv("TopGames.csv")
TopGames

,App Name,App Id,Category,Rating,Rating Count,Released,Content Rating
2068486,8 Ball Pool,com.miniclip.eightballpool,Sports,4.4,21754025.0,"Jan 23, 2013",Everyone
99040,Among Us,com.innersloth.spacemafia,Action,3.5,12022640.0,"Jun 15, 2018",Everyone 10+
1459273,Brawl Stars,com.supercell.brawlstars,Action,4.3,18066559.0,"Dec 12, 2018",Everyone 10+
2297151,Call of Duty®,com.activision.callofduty.shooter,Action,4.5,13629457.0,"Sep 30, 2019",Mature 17+
286345,Candy Crush Saga,com.king.candycrushsaga,Casual,4.6,31476637.0,"Nov 15, 2012",Everyone
1433519,Clash Royale,com.supercell.clashroyale,Strategy,4.2,31018623.0,"Mar 1, 2016",Everyone 10+
423997,Clash of Clans,com.supercell.clashofclans,Strategy,4.5,56025424.0,"Sep 30, 2013",Everyone 10+
1048947,Dream League Soccer,com.firsttouchgames.dls3,Sports,4.5,13387156.0,"Apr 7, 2016",Everyone
1119750,Gardenscapes,com.playrix.gardenscapes,Casual,4.3,10756538.0,"Aug 24, 2016",Everyone
244319,Garena Free Fire,com.dts.freefireth,Action,4.2,89177097.0,"Dec 7, 2017",Mature 17+


In [5]:
print('SOURCE |      GAME NAME      | REVIEWS')
print('======================================')
total_review = 0
for index, game in TopGames.iterrows():
    language = 'en'
    country = 'us'
    app_id=game['App Id']
    file_path = f"GameReviewData{country.upper()}/{game['App Name'].replace(' ', '')}-{language.upper()}-{country.upper()}.csv"
    total_review = total_review + int(TopGames.index.size)

    if os.path.exists(file_path):
        source = 'Local'
        data = pd.read_csv(file_path, low_memory=False)
        print(f"{source:<7}| {game['App Name']:<20}| {data.index.size:>7,}")
    else:
        source = 'Scrape'
        print(f"{source:<7}| {game['App Name']:<20}| ", end="")
        result = reviews_all(
        app_id,
        sleep_milliseconds=0,
        lang='en',
        country='us',
        sort=Sort.MOST_RELEVANT,
        )
        data = pd.DataFrame.from_dict(result)
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        data.to_csv(file_path)
        print(f"{data.index.size:>7,}")

print(f"\nSuccesfully loaded {total_review:,} reviews from {TopGames.index.size:,} games")

SOURCE |      GAME NAME      | REVIEWS
Local  | 8 Ball Pool         | 126,000
Local  | Among Us            |  72,000
Local  | Brawl Stars         |  49,500
Local  | Call of Duty®       |  85,500
Local  | Candy Crush Saga    |  81,000
Local  | Clash Royale        |  85,500
Local  | Clash of Clans      |  63,000
Local  | Dream League Soccer | 144,000
Local  | Gardenscapes        | 103,500
Local  | Garena Free Fire    |  72,000
Local  | Homescapes          |  90,000
Local  | Minion Rush         | 301,500
Local  | Mobile Legends      | 139,500
Local  | My Talking Angela   |  76,500
Local  | My Talking Tom      |  85,500
Local  | PUBG MOBILE         |  85,500
Local  | Pokémon GO          |  94,500
Local  | Pou                 | 117,000
Local  | Roblox              |  63,000
Local  | Shadow Fight 2      | 108,000

Succesfully loaded 400 reviews from 20 games


In [6]:
folder = "GameReviewDataUS"
dfs = []

for _, game in TopGames.iterrows():
    safe_name = game["App Name"].replace(" ", "")
    file_path = f"{folder}/{safe_name}-EN-US.csv"

    df = pd.read_csv(file_path, usecols=["content", "score", "at"], low_memory=False).dropna()
    df["game_name"] = game["App Name"]
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)[["game_name", "content", "score", "at"]]
data["year"] = pd.to_datetime(data["at"], errors="coerce").dt.year
data.to_csv("TopGamesData.csv", index=False)
data.head()

,game_name,content,score,at,year
0,8 Ball Pool,After having an account that I spent actual mo...,2,2025-10-24 23:57:54,2025
1,8 Ball Pool,"Fun game, but has WAY too many ads to try to t...",1,2025-10-29 04:41:27,2025
2,8 Ball Pool,I quit playing this for several years despite ...,2,2025-10-28 14:19:35,2025
3,8 Ball Pool,I love this game but it is too easy to select ...,3,2025-10-29 12:04:48,2025
4,8 Ball Pool,"The game itself is fun and well-designed, but ...",1,2025-09-29 23:40:05,2025
